In [1]:
import pandas as pd

In [3]:
new_df=pd.read_csv('suggestions.csv')

In [4]:
from pymystem3 import Mystem
m=Mystem()

In [5]:
import operator
def suggest(input_string, collocations):
    c_max=max(collocations.pmi)
    suggestions={}
    w1=input_string.split()[0]
    w2=input_string.split()[1]
    if w1 not in list(collocations.first_word) and w2 not in list(collocations.second_word):
        pass
    else:
        tags=[]
        if w1 not in list(collocations.first_word):
            w2_ind=list(collocations.second_word).index(w2)
            tags.append(m.analyze(w1)[0]['analysis'][0]['gr'].split(',')[0])
            tags.append(collocations.second_tag[w2_ind])
            for i in range(len(collocations)-1):
                if collocations.second_word[i]==w2 and collocations.first_tag[i]==tags[0]:
                    suggestions[collocations.first_word[i]+ ' ' +w2]=collocations.pmi[i]/c_max
                    #suggestions[collocations.first_word[i]+ ' ' +w2]+=model.wv.similarity(collocations.first_word[i], w2)
        elif w2 not in list(collocations.second_word):
            w1_ind=list(collocations.first_word).index(w1)
            tags.append(collocations.first_tag[w1_ind])
            tags.append(m.analyze(w2)[0]['analysis'][0]['gr'].split(',')[0])
            for i in range(len(collocations)-1):
                if collocations.first_word[i]==w1 and collocations.second_tag[i]==tags[1]:
                    suggestions[w1+ ' ' +collocations.second_word[i]]=collocations.pmi[i]/c_max
                    #suggestions[collocations.first_word[i]+ ' ' +w2]+=model.wv.similarity(w1, collocations.second_word[i])
        else:
            w1_sub=[]
            w2_sub=[]
            w1_ind=list(collocations.first_word).index(w1)
            w2_ind=list(collocations.second_word).index(w2)
            tags.append(collocations.first_tag[w1_ind])
            tags.append(collocations.second_tag[w2_ind])
            for i in range(len(collocations)-1):
                if collocations.first_word[i]==w1 and collocations.second_tag[i]==tags[1]:
                    suggestions[w1+ ' ' +collocations.second_word[i]]=collocations.pmi[i]/c_max
                    #suggestions[collocations.first_word[i]+ ' ' +w2]+=model.wv.similarity(w1, collocations.second_word[i])
                    w2_sub.append(collocations.second_word[i])
                if collocations.second_word[i]==w2 and collocations.first_tag[i]==tags[0]:
                    suggestions[collocations.first_word[i]+ ' ' +w2]=collocations.pmi[i]/c_max
                    #suggestions[collocations.first_word[i]+ ' ' +w2]+=model.wv.similarity(collocations.first_word[i], w2)
                    w1_sub.append(collocations.first_word[i])
            for w in w1_sub:
                for j in range(len(collocations)-1):
                    if collocations.first_word[j]==w and collocations.second_word[j] in w2_sub:
                        suggestions[w+' '+w2]+=1.0/len(w2_sub)
            for k in w2_sub:
                for j in range(len(collocations)-1):
                    if collocations.second_word[j]==k and collocations.first_word[j] in w1_sub:
                        suggestions[w1+' '+k]+=1.0/len(w1_sub)
    if len(suggestions)>10:
        return sorted(suggestions.items(), key=operator.itemgetter(1))[-10:]
    else:
        return sorted(suggestions.items(), key=operator.itemgetter(1))

In [6]:
suggest('автор думает', new_df)

[('автор выделяет', 0.54247529802396188),
 ('автор предлагает', 0.56440546951472503),
 ('автор выражает', 0.5659547808523554),
 ('автор придерживается', 0.57419794167165827),
 ('автор публикует', 0.605920585319349),
 ('автор апеллирует', 0.66015080282660032),
 ('автор наталкивается', 0.66015080282660032),
 ('автор посвящает', 0.66015080282660032),
 ('автор сосредоточивает', 0.66015080282660032),
 ('автор стремился', 0.66015080282660032)]

In [7]:
suggest('автор котик', new_df)

[('автор слово', 0.22816458845353194),
 ('автор данных', 0.26607138814595011),
 ('автор слова', 0.26607138814595011),
 ('автор статьи', 0.32664867892126986),
 ('автор юшманов', 0.33600161247347426),
 ('автор эксперимента', 0.39536400824943946),
 ('автор игоревой', 0.42978476981751834),
 ('автор оригинала', 0.42978476981751834),
 ('автор реплики', 0.43401486300945918),
 ('автор открытия', 0.605920585319349)]

In [8]:
suggest('котик предлагает', new_df)

[('схема предлагает', 0.49071081748177076),
 ('ягич предлагает', 0.54013967186800538),
 ('автор предлагает', 0.56440546951472503),
 ('исследователь предлагает', 0.59818711793699597),
 ('фалилеев предлагает', 0.69238313991124101)]

In [9]:
suggest('анализ апеллирует', new_df)

[('анализ начинается', 0.4425974502434144),
 ('анализ осуществлялся', 0.48858450693136551),
 ('анализ показал', 0.51285030457808689),
 ('анализ производился', 0.5707201395245689),
 ('автор апеллирует', 0.86015080282660028),
 ('анализ показывает', 1.4138275217660417)]

In [10]:
suggest('котик котик', new_df)

[]